# Tree Methods Code Along

In this lecture we will code along with some data and test out 3 different tree methods:

* A single decision tree
* A random forest
* A gradient boosted tree classifier
    
We will be using a college dataset to try to classify colleges as Private or Public based off these features:

    Private A factor with levels No and Yes indicating private or public university
    Apps Number of applications received
    Accept Number of applications accepted
    Enroll Number of new students enrolled
    Top10perc Pct. new students from top 10% of H.S. class
    Top25perc Pct. new students from top 25% of H.S. class
    F.Undergrad Number of fulltime undergraduates
    P.Undergrad Number of parttime undergraduates
    Outstate Out-of-state tuition
    Room.Board Room and board costs
    Books Estimated book costs
    Personal Estimated personal spending
    PhD Pct. of faculty with Ph.D.’s
    Terminal Pct. of faculty with terminal degree
    S.F.Ratio Student/faculty ratio
    perc.alumni Pct. alumni who donate
    Expend Instructional expenditure per student
    Grad.Rate Graduation rate

In [2]:
import findspark
findspark.init('/home/cheng/spark-2.3.0-bin-hadoop2.7')
import pyspark

In [3]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treecode').getOrCreate()

In [4]:
# Load training data
data = spark.read.csv('College.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

### Spark Formatting of Data

In [7]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [10]:
output = assembler.transform(data)

Deal with Private column being "yes" or "no"

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [13]:
final_data = output_fixed.select("features",'PrivateIndex')

In [14]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [15]:
train_data.head(2)

[Row(features=DenseVector([81.0, 72.0, 51.0, 33.0, 71.0, 139.0, 3.0, 8730.0, 3600.0, 400.0, 800.0, 92.0, 92.0, 9.3, 17.0, 10922.0, 58.0]), PrivateIndex=0.0),
 Row(features=DenseVector([150.0, 130.0, 88.0, 23.0, 50.0, 341.0, 768.0, 10300.0, 4130.0, 500.0, 1700.0, 44.0, 58.0, 10.2, 37.0, 9678.0, 75.0]), PrivateIndex=0.0)]

### The Classifiers

In [16]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

Create all three models:

In [17]:
# Use mostly defaults to make this comparison "fair"

dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

Train all three models:

In [18]:
# Train the models (its three models, so it might take some time)
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Model Comparison

Let's compare each of these models!

In [19]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

In [24]:
dtc_predictions.head(3)

[Row(features=DenseVector([100.0, 90.0, 35.0, 10.0, 52.0, 282.0, 331.0, 8400.0, 2812.0, 300.0, 2134.0, 10.0, 50.0, 12.1, 24.0, 7976.0, 52.0]), PrivateIndex=0.0, rawPrediction=DenseVector([78.0, 1.0]), probability=DenseVector([0.9873, 0.0127]), prediction=0.0),
 Row(features=DenseVector([141.0, 118.0, 55.0, 12.0, 21.0, 201.0, 173.0, 8300.0, 4850.0, 450.0, 1300.0, 53.0, 53.0, 9.5, 19.0, 6936.0, 76.0]), PrivateIndex=0.0, rawPrediction=DenseVector([230.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(features=DenseVector([152.0, 128.0, 75.0, 17.0, 41.0, 282.0, 22.0, 9100.0, 3700.0, 500.0, 1400.0, 48.0, 48.0, 8.5, 26.0, 8960.0, 50.0]), PrivateIndex=0.0, rawPrediction=DenseVector([78.0, 1.0]), probability=DenseVector([0.9873, 0.0127]), prediction=0.0)]

**Evaluation Metrics:**

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", 
                                                  predictionCol="prediction", metricName="accuracy")

In [22]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)

In [23]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 92.63%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 96.77%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 94.47%


Interesting! Optional Assignment - play around with the parameters of each of these models, can you squeeze some more accuracy out of them? Or is the data the limiting factor?